#code aimed at generating a content based recommmender system
#using the Xwines dataset

In [2]:
##standard imports
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix as csr
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from matplotlib import pyplot as plt
import cv2 as cv
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
import scipy.sparse as sp
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)


In [18]:
##importing the data
user_reviews = pd.read_csv("C:\\Users\\Dell\\reommenders\\RecommenderSystems\\XWines_Slim_150K_ratings.csv", low_memory=False, encoding="utf-8", memory_map=True)
wine_data= pd.read_csv("C:\\Users\\Dell\\reommenders\\RecommenderSystems\\XWines_Slim_1k_wines.csv", low_memory=False, encoding="utf-8", memory_map=True)

#define scalar and encoder
encoder = OneHotEncoder()
scaler = MinMaxScaler()

In [19]:
# Preprocessing for 'Grapes', 'Harmonize', and 'Vintages'
wine_data['Grapes'] = wine_data['Grapes'].apply(lambda x: ' '.join(eval(x)))  # Convert list to space-separated string
wine_data['Harmonize'] = wine_data['Harmonize'].apply(lambda x: ' '.join(eval(x)))




# Function to handle mixed vintages: list or integer
def convert_to_list(vintages):
    if isinstance(vintages, int):
        # If the vintage is a single integer, wrap it in a list
        return [str(vintages)]
    elif isinstance(vintages, str):
        # If it's a string, evaluate it as a list (since strings in CSV could be saved as "['1990', '1995']")
        try:
            return list(map(str, eval(vintages)))  # Convert all elements to strings
        except:
            return [str(vintages)]  # In case it's a string number
    else:
        # In case it's an unexpected format, treat as string
        return [str(vintages)]

# Apply the conversion to the 'Vintages' column
wine_data['Vintages'] = wine_data['Vintages'].apply(convert_to_list)

# Convert the list into a space-separated string
wine_data['Vintages'] = wine_data['Vintages'].apply(lambda x: ' '.join(x))


In [20]:

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
import numpy as np

# TF-IDF for 'Grapes', 'Harmonize', and 'Vintages'
tfidf = TfidfVectorizer()

wine_data_tfidf_grapes = tfidf.fit_transform(wine_data['Grapes'])
wine_data_tfidf_harmonize = tfidf.fit_transform(wine_data['Harmonize'])
wine_data_tfidf_vintages = tfidf.fit_transform(wine_data['Vintages'])

# One-hot encoding for 'Type', 'Body', 'Acidity', 'Country', 'RegionName', 'WineryName'
one_hot = OneHotEncoder()

categorical_features = ['Type', 'Body', 'Acidity', 'Country', 'RegionName', 'WineryName']
wine_categorical_onehot = one_hot.fit_transform(wine_data[categorical_features])

# Scale the numerical feature 'ABV'
scaler = StandardScaler()
wine_abv_scaled = scaler.fit_transform(wine_data[['ABV']])



In [34]:
# Define the weights for different features
weights ={'grapes': 2.6351178844772396, 'harmonize': 1.553630179215512, 'vintages': 1.760447782530082, 'type': 2.5859874914949206, 'body': 1.2628975201148853, 'acidity': 0.9828619145406399, 'country': 0.6956261879697497, 'region': 0.6957519257816862, 'winery': 0.7668374998938243, 'abv': 2.11521395128059}

# Apply weights to categorical features (by columns)
wine_type_onehot = wine_categorical_onehot[:, :one_hot.categories_[0].size] * weights['type']  # 'Type' weight
wine_body_onehot = wine_categorical_onehot[:, one_hot.categories_[0].size:one_hot.categories_[0].size + one_hot.categories_[1].size] * weights['body']
wine_acidity_onehot = wine_categorical_onehot[:, one_hot.categories_[0].size + one_hot.categories_[1].size:one_hot.categories_[0].size + one_hot.categories_[1].size + one_hot.categories_[2].size] * weights['acidity']
wine_country_onehot = wine_categorical_onehot[:, one_hot.categories_[0].size + one_hot.categories_[1].size + one_hot.categories_[2].size:one_hot.categories_[0].size + one_hot.categories_[1].size + one_hot.categories_[2].size + one_hot.categories_[3].size] * weights['country']
wine_region_onehot = wine_categorical_onehot[:, one_hot.categories_[0].size + one_hot.categories_[1].size + one_hot.categories_[2].size + one_hot.categories_[3].size:one_hot.categories_[0].size + one_hot.categories_[1].size + one_hot.categories_[2].size + one_hot.categories_[3].size + one_hot.categories_[4].size] * weights['region']
wine_winery_onehot = wine_categorical_onehot[:, one_hot.categories_[0].size + one_hot.categories_[1].size + one_hot.categories_[2].size + one_hot.categories_[3].size + one_hot.categories_[4].size:] * weights['winery']

# Scale the numerical feature 'ABV'
scaler = StandardScaler()
wine_abv_scaled = scaler.fit_transform(wine_data[['ABV']]) * weights['abv']  # Apply weight for 'ABV'


wine_data_tfidf_grapes = wine_data_tfidf_grapes * weights['grapes']  # Apply weight for 'Grapes'
wine_data_tfidf_harmonize = wine_data_tfidf_harmonize * weights['harmonize']  # Apply weight for 'Harmonize'
wine_data_tfidf_vintages = wine_data_tfidf_vintages * weights['vintages']  # Apply weight for 'Vintages'


In [35]:
wine_features = sp.hstack([
    wine_data_tfidf_grapes, 
    wine_data_tfidf_harmonize, 
    wine_data_tfidf_vintages,
    wine_type_onehot, 
    wine_body_onehot, 
    wine_acidity_onehot,
    wine_country_onehot, 
    wine_region_onehot, 
    wine_winery_onehot, 
    wine_abv_scaled
])

In [36]:
from sklearn.metrics.pairwise import cosine_similarity

# Compute cosine similarity between all wines
wine_similarity = cosine_similarity(wine_features)

# Store this in a DataFrame for easier access
wine_similarity_df = pd.DataFrame(wine_similarity, index=wine_data['WineID'], columns=wine_data['WineID'])

print(wine_similarity_df.head()) 

WineID    100001    100002    100003    100005    100007    100008    100010  \
WineID                                                                         
100001  1.000000  0.263431  0.213276  0.262206  0.927335  0.065622 -0.025646   
100002  0.263431  1.000000  0.454260  0.515686  0.239195  0.417681  0.347340   
100003  0.213276  0.454260  1.000000  0.816976  0.213276  0.942164  0.487995   
100005  0.262206  0.515686  0.816976  1.000000  0.262206  0.756811  0.403824   
100007  0.927335  0.239195  0.213276  0.262206  1.000000  0.065622 -0.025646   

WineID    100012    100013    100014  ...    198580    198885    199204  \
WineID                                ...                                 
100001  0.401144 -0.425455 -0.425455  ...  0.142510  0.133567  0.192011   
100002  0.110867 -0.137383 -0.137383  ...  0.098452  0.092556  0.046022   
100003  0.155324 -0.098496 -0.098496  ...  0.063788  0.040321  0.097923   
100005  0.185812 -0.230253 -0.230253  ...  0.017666  0.117788  0

In [37]:
def recommend_wines(user_id, user_reviews, wine_similarity_df, top_n=5,returnall=False):
    # Get the wines the user has rated
    user_wines = user_reviews[user_reviews['UserID'] == user_id]
    
    # Filter for wines the user rated 4 or 5
    high_rated_wines = user_wines[user_wines['Rating'] >= 4]['WineID']
    
    # Get similarity scores for each of these wines
    similar_wines = pd.Series(dtype=float)
    
    for wine_id in high_rated_wines:
        similar_wines = similar_wines.append(wine_similarity_df[wine_id])
    
    # Sort the wines by similarity score
    similar_wines = similar_wines.groupby(similar_wines.index).mean().sort_values(ascending=False)
    
    # Exclude wines the user has already rated
    similar_wines = similar_wines[~similar_wines.index.isin(user_wines['WineID'])]
    
    if returnall:
        return similar_wines.index.tolist()
    return similar_wines.head(top_n).index.tolist()

# Example recommendation for user with ID 1
recommended_wines = recommend_wines(1356810, user_reviews, wine_similarity_df)
print(recommended_wines)

[115984, 116995, 179690, 111448, 114606]


In [40]:
from sklearn.metrics import precision_score, recall_score, f1_score
from random import random

def evaluate_recommender(user_reviews, wine_similarity_df, top_n=5,threshold=4):
    all_precision_scores = []
    all_recall_scores = []
    all_f1_scores = []

    # Loop through each user in the dataset
    unique=user_reviews['UserID'].unique()
    TP=1
    FP=1
    FN=1
    for user_id in unique:
        # Get all the wines the user has rated 4 or 5
        user_ratings = user_reviews[user_reviews['UserID'] == user_id]
        
        if len(user_ratings) <30:
            continue
        # we want to evaluate for new users
        # so we want to simulate users with few ratings

        # select 10 random ratings
        sparse = user_ratings.sample(10)
        # remove the selected ratings from the user_ratings
        remaining = user_ratings.drop(sparse.index)

        # now we want to evaluate the recommender on the sparse ratings
        # if it recommends wines with a rating of 4 or 5, it's a hit
        recommended_wines = recommend_wines(user_id, sparse, wine_similarity_df, top_n) 
        
        # find wines that the user has rated 
        reviewed_recommended = remaining[remaining['WineID'].isin(recommended_wines)]
        for index, row in reviewed_recommended.iterrows():
            if row['Rating'] >= threshold:
                TP+=1
            else:
                FP+=1
        ## now we want the amount of false negatives
        ## this is the amount of wines that the user rated 4 or 5
        




        
            

    print("TP:"+str(TP))
    print("FP:"+str(FP))
    print("FN:"+str(FN))
    precision=TP/(TP+FP)
    recall=TP/(TP+FN)
    f1=2*(precision*recall)/(precision+recall)
    # Return the mean precision, recall, and F1-score across all users
    return {
        'precision': precision,
        'recall': recall,
        'f1': f1
    }

print(evaluate_recommender(user_reviews,wine_similarity_df,15,threshold=3.5))

TP:358
FP:47
FN:1
{'precision': 0.8839506172839506, 'recall': 0.9972144846796658, 'f1': 0.93717277486911}


block combines all into 1

In [31]:
def evaluate_weights(weights):
    
    weights = {
        'grapes': weights[0],
        'harmonize': weights[1],
        'vintages': weights[2],
        'type': weights[3],
        'body': weights[4],
        'acidity': weights[5],
        'country': weights[6],     # A bit more important
        'region': weights[7],
        'winery': weights[8],      # Slightly less important
        'abv': weights[9]          # Least important
    }

    tfidf = TfidfVectorizer()

    wine_data_tfidf_grapes = tfidf.fit_transform(wine_data['Grapes'])
    wine_data_tfidf_harmonize = tfidf.fit_transform(wine_data['Harmonize'])
    wine_data_tfidf_vintages = tfidf.fit_transform(wine_data['Vintages'])

    # One-hot encoding for 'Type', 'Body', 'Acidity', 'Country', 'RegionName', 'WineryName'
    one_hot = OneHotEncoder()

    categorical_features = ['Type', 'Body', 'Acidity', 'Country', 'RegionName', 'WineryName']
    wine_categorical_onehot = one_hot.fit_transform(wine_data[categorical_features])

    # Scale the numerical feature 'ABV'
    scaler = StandardScaler()
    wine_abv_scaled = scaler.fit_transform(wine_data[['ABV']])
    # Apply weights to categorical features (by columns)
    wine_type_onehot = wine_categorical_onehot[:, :one_hot.categories_[0].size] * weights['type']  # 'Type' weight
    wine_body_onehot = wine_categorical_onehot[:, one_hot.categories_[0].size:one_hot.categories_[0].size + one_hot.categories_[1].size] * weights['body']
    wine_acidity_onehot = wine_categorical_onehot[:, one_hot.categories_[0].size + one_hot.categories_[1].size:one_hot.categories_[0].size + one_hot.categories_[1].size + one_hot.categories_[2].size] * weights['acidity']
    wine_country_onehot = wine_categorical_onehot[:, one_hot.categories_[0].size + one_hot.categories_[1].size + one_hot.categories_[2].size:one_hot.categories_[0].size + one_hot.categories_[1].size + one_hot.categories_[2].size + one_hot.categories_[3].size] * weights['country']
    wine_region_onehot = wine_categorical_onehot[:, one_hot.categories_[0].size + one_hot.categories_[1].size + one_hot.categories_[2].size + one_hot.categories_[3].size:one_hot.categories_[0].size + one_hot.categories_[1].size + one_hot.categories_[2].size + one_hot.categories_[3].size + one_hot.categories_[4].size] * weights['region']
    wine_winery_onehot = wine_categorical_onehot[:, one_hot.categories_[0].size + one_hot.categories_[1].size + one_hot.categories_[2].size + one_hot.categories_[3].size + one_hot.categories_[4].size:] * weights['winery']

    # Scale the numerical feature 'ABV'
    scaler = StandardScaler()
    wine_abv_scaled = scaler.fit_transform(wine_data[['ABV']]) * weights['abv']  # Apply weight for 'ABV'


    wine_data_tfidf_grapes = wine_data_tfidf_grapes * weights['grapes']  # Apply weight for 'Grapes'
    wine_data_tfidf_harmonize = wine_data_tfidf_harmonize * weights['harmonize']  # Apply weight for 'Harmonize'
    wine_data_tfidf_vintages = wine_data_tfidf_vintages * weights['vintages']  # Apply weight for 'Vintages'
    wine_features = sp.hstack([
        wine_data_tfidf_grapes, 
        wine_data_tfidf_harmonize, 
        wine_data_tfidf_vintages,
        wine_type_onehot, 
        wine_body_onehot, 
        wine_acidity_onehot,
        wine_country_onehot, 
        wine_region_onehot, 
        wine_winery_onehot, 
        wine_abv_scaled
    ])
    # Compute cosine similarity between all wines
    wine_similarity = cosine_similarity(wine_features)

    # Store this in a DataFrame for easier access
    wine_similarity_df = pd.DataFrame(wine_similarity, index=wine_data['WineID'], columns=wine_data['WineID'])

    values=evaluate_recommender(user_reviews,wine_similarity_df)
    return values['precision']


def minimise(weights):
    return -evaluate_weights(weights)




Next part is tweaking the weigths

In [33]:
from pyswarm import pso
import numpy as np
lower_bounds = [0.1] * 10  # Minimum weight is 0.1
upper_bounds = [3.0] * 10  # Maximum weight is 3.0


initial_weights = [2.0, 1.5, 0.8, 1.0, 1.0, 1.0, 1.2, 1.0, 0.9, 0.5]
# Run PSO optimization
best_weights, best_score = pso(minimise, lower_bounds, upper_bounds, swarmsize=20, maxiter=10)

# Print the optimal weights found
optimal_weights = {
    'grapes': best_weights[0],
    'harmonize': best_weights[1],
    'vintages': best_weights[2],
    'type': best_weights[3],
    'body': best_weights[4],
    'acidity': best_weights[5],
    'country': best_weights[6],
    'region': best_weights[7],
    'winery': best_weights[8],
    'abv': best_weights[9]
}

print(f"Optimal Weights: {optimal_weights}")
print(f"Best Evaluation Score: {best_score}")






TP:135
FP:51
FN:1
TP:193
FP:74
FN:1
TP:208
FP:60
FN:1
TP:151
FP:57
FN:1
TP:163
FP:57
FN:1
TP:189
FP:61
FN:1
TP:160
FP:56
FN:1
TP:151
FP:48
FN:1
TP:178
FP:60
FN:1
TP:208
FP:60
FN:1
TP:178
FP:44
FN:1
TP:177
FP:65
FN:1
TP:163
FP:40
FN:1
TP:165
FP:55
FN:1
TP:207
FP:66
FN:1
TP:193
FP:64
FN:1
TP:173
FP:61
FN:1
TP:198
FP:53
FN:1
TP:160
FP:53
FN:1
TP:185
FP:55
FN:1
TP:158
FP:67
FN:1
TP:176
FP:69
FN:1
TP:187
FP:56
FN:1
TP:181
FP:54
FN:1
TP:150
FP:60
FN:1
TP:213
FP:67
FN:1
TP:118
FP:57
FN:1
TP:152
FP:60
FN:1
TP:190
FP:48
FN:1
TP:199
FP:56
FN:1
TP:145
FP:52
FN:1
TP:166
FP:61
FN:1
TP:135
FP:37
FN:1
TP:161
FP:66
FN:1
TP:210
FP:62
FN:1
TP:186
FP:61
FN:1
TP:186
FP:58
FN:1
TP:202
FP:52
FN:1
TP:148
FP:46
FN:1
TP:185
FP:66
FN:1
TP:174
FP:47
FN:1
TP:181
FP:55
FN:1
TP:187
FP:72
FN:1
TP:214
FP:53
FN:1
TP:145
FP:41
FN:1
TP:201
FP:68
FN:1
TP:126
FP:49
FN:1
TP:193
FP:63
FN:1
TP:169
FP:60
FN:1
TP:169
FP:49
FN:1
TP:102
FP:53
FN:1
TP:195
FP:71
FN:1
TP:117
FP:48
FN:1
TP:161
FP:50
FN:1
TP:212
FP:64
FN:1
TP:153
FP:

In [46]:
##now we extend the model by making it a hybrid model with a collaborative filtering model
import pandas as pd
from scipy.sparse import csr_matrix
from implicit.als import AlternatingLeastSquares
from implicit.evaluation import mean_average_precision_at_k

# Load the CSV file
# Make sure the file path is correct

# Display the first few rows of the DataFrame
df=user_reviews

# Select the relevant columns (UserID, WineID, Rating)
df_filtered = df[['UserID', 'WineID', 'Rating']]

# Create a user-item interaction matrix (rows = UserID, columns = WineID, values = Rating)
user_item_matrix = csr_matrix((df_filtered['Rating'], (df_filtered['UserID'], df_filtered['WineID'])))

# Initialize the ALS model and fit it using the user-item matrix
model = AlternatingLeastSquares(factors=100, regularization=0.01, iterations=50)
model.fit(user_item_matrix)



100%|██████████| 50/50 [01:23<00:00,  1.67s/it]


In [50]:
#now predict the ratings for the user
user_id = 1356810
# Get the user's ratings
user_ratings = df_filtered[df_filtered['UserID'] == user_id]

# Predict ratings for all wines
user_wine_scores = model.recommend(user_id, user_item_matrix[user_id], N=10)
print(user_wine_scores)


(array([113288, 113321, 113600, 111479, 167439, 115365, 114606, 113344,
       167463, 115984]), array([0.60073334, 0.47158247, 0.45213318, 0.41883025, 0.37962162,
       0.3680351 , 0.35787725, 0.33687162, 0.33371258, 0.3297872 ],
      dtype=float32))


In [52]:
#lets make a class for the models
class contentBased:
    def __init__(self,user_reviews,wine_data):
        self.user_reviews=user_reviews
        self.wine_data=wine_data
        self.weights ={'grapes': 2.6351178844772396, 'harmonize': 1.553630179215512, 'vintages': 1.760447782530082, 'type': 2.5859874914949206, 'body': 1.2628975201148853, 'acidity': 0.9828619145406399, 'country': 0.6956261879697497, 'region': 0.6957519257816862, 'winery': 0.7668374998938243, 'abv': 2.11521395128059}
        
    def preprocess(self):
        # Preprocessing for 'Grapes', 'Harmonize', and 'Vintages'
        self.wine_data['Grapes'] = self.wine_data['Grapes'].apply(lambda x: ' '.join(eval(x)))
        self.wine_data['Harmonize'] = self.wine_data['Harmonize'].apply(lambda x: ' '.join(eval(x)))
        # Function to handle mixed vintages: list or integer
        def convert_to_list(vintages):
            if isinstance(vintages, int):
                # If the vintage is a single integer, wrap it in a list
                return [str(vintages)]
            elif isinstance(vintages, str):
                # If it's a string, evaluate it as a list (since strings in CSV could be saved as "['1990', '1995']")
                try:
                    return list(map(str, eval(vintages)))  # Convert all elements to strings
                except:
                    return [str(vintages)]
            else:
                return [str(vintages)]
        # Apply the conversion to the 'Vintages' column
        self.wine_data['Vintages'] = self.wine_data['Vintages'].apply(convert_to_list)
        # Convert the list into a space-separated string
        self.wine_data['Vintages'] = self.wine_data['Vintages'].apply(lambda x: ' '.join(x))
        # TF-IDF for 'Grapes', 'Harmonize', and 'Vintages'
        tfidf = TfidfVectorizer()
        self.wine_data_tfidf_grapes = tfidf.fit_transform(self.wine_data['Grapes'])
        self.wine_data_tfidf_harmonize = tfidf.fit_transform(self.wine_data['Harmonize'])
        self.wine_data_tfidf_vintages = tfidf.fit_transform(self.wine_data['Vintages'])
        # One-hot encoding for 'Type', 'Body', 'Acidity', 'Country', 'RegionName', 'WineryName'
        one_hot = OneHotEncoder()
        categorical_features = ['Type', 'Body', 'Acidity', 'Country', 'RegionName', 'WineryName']
        self.wine_categorical_onehot = one_hot.fit_transform(self.wine_data[categorical_features])
        # Scale the numerical feature 'ABV'
        scaler = StandardScaler()
        self.wine_abv_scaled = scaler.fit_transform(self.wine_data[['ABV']])
        # Apply weights to categorical features (by columns)
        self.wine_type_onehot = self.wine_categorical_onehot[:, :one_hot.categories_[0].size] * self.weights['type']
        self.wine_body_onehot = self.wine_categorical_onehot[:, one_hot.categories_[0].size:one_hot.categories_[0].size + one_hot.categories_[1].size] * self.weights['body']
        self.wine_acidity_onehot = self.wine_categorical_onehot[:, one_hot.categories_[0].size + one_hot.categories_[1].size:one_hot.categories_[0].size + one_hot.categories_[1].size + one_hot.categories_[2].size] * self.weights['acidity']
        self.wine_country_onehot = self.wine_categorical_onehot[:, one_hot.categories_[0].size + one_hot.categories_[1].size + one_hot.categories_[2].size:one_hot.categories_[0].size + one_hot.categories_[1].size + one_hot.categories_[2].size + one_hot.categories_[3].size] * self.weights['country']
        self.wine_region_onehot = self.wine_categorical_onehot[:, one_hot.categories_[0].size + one_hot.categories_[1].size + one_hot.categories_[2].size + one_hot.categories_[3].size:one_hot.categories_[0].size + one_hot.categories_[1].size + one_hot.categories_[2].size + one_hot.categories_[3].size + one_hot.categories_[4].size] * self.weights['region']
        self.wine_winery_onehot = self.wine_categorical_onehot[:, one_hot.categories_[0].size + one_hot.categories_[1].size + one_hot.categories_[2].size + one_hot.categories_[3].size + one_hot.categories_[4].size:] * self.weights['winery']
        # Scale the numerical feature 'ABV'
        scaler = StandardScaler()
        self.wine_abv_scaled = scaler.fit_transform(self.wine_data[['ABV']]) * self.weights['abv']
        self.wine_data_tfidf_grapes = self.wine_data_tfidf_grapes * self.weights['grapes']
        self.wine_data_tfidf_harmonize = self.wine_data_tfidf_harmonize * self.weights['harmonize']
        self.wine_data_tfidf_vintages = self.wine_data_tfidf_vintages * self.weights['vintages']
        self.wine_features = sp.hstack([
            self.wine_data_tfidf_grapes,
            self.wine_data_tfidf_harmonize,
            self.wine_data_tfidf_vintages,
            self.wine_type_onehot,
            self.wine_body_onehot,
            self.wine_acidity_onehot,
            self.wine_country_onehot,
            self.wine_region_onehot,
            self.wine_winery_onehot,
            self.wine_abv_scaled
        ])
        # Compute cosine similarity between all wines
        self.wine_similarity = cosine_similarity(self.wine_features)
        # Store this in a DataFrame for easier access
        self.wine_similarity_df = pd.DataFrame(self.wine_similarity, index=self.wine_data['WineID'], columns=self.wine_data['WineID'])
    def recommend_wines(self,user_id, top_n=5,returnall=False):
        # Get the wines the user has rated
        user_wines = self.user_reviews[self.user_reviews['UserID'] == user_id]
        # Filter for wines the user rated 4 or 5
        high_rated_wines = user_wines[user_wines['Rating'] >= 4]['WineID']
        # Get similarity scores for each of these wines
        similar_wines = pd.Series(dtype=float)
        for wine_id in high_rated_wines:
            similar_wines = similar_wines.append(self.wine_similarity_df[wine_id])
        # Sort the wines by similarity score
        similar_wines = similar_wines.groupby(similar_wines.index).mean().sort_values(ascending=False)
        # Exclude wines the user has already rated
        similar_wines = similar_wines[~similar_wines.index.isin(user_wines['WineID'])]
        if returnall:
            return similar_wines.index.tolist()
        return similar_wines.head(top_n).index.tolist()
    
class collaborativeFiltering:
    def __init__(self,user_reviews):
        self.user_reviews=user_reviews
        self.df_filtered = self.user_reviews[['UserID', 'WineID', 'Rating']]
        self.user_item_matrix = csr_matrix((self.df_filtered['Rating'], (self.df_filtered['UserID'], self.df_filtered['WineID'])))
        self.model = AlternatingLeastSquares(factors=100, regularization=0.01, iterations=50)
        self.model.fit(self.user_item_matrix)
    def recommend_wines(self,user_id, top_n=5,returnall=False):
        # Get the user's ratings
        user_ratings = self.df_filtered[self.df_filtered['UserID'] == user_id]
        # Predict ratings for all wines
        user_wine_scores = self.model.recommend(user_id, self.user_item_matrix[user_id], N=10)
        if returnall:
            return user_wine_scores
        return user_wine_scores.head(top_n).index.tolist()
    
class hybridModel:

    def __init__(self,user_reviews,wine_data,ratio):
        self.user_reviews=user_reviews
        self.wine_data=wine_data
        self.contentBasedModel=contentBased(self.user_reviews,self.wine_data)
        self.collaborativeFilteringModel=collaborativeFiltering(self.user_reviews)
        self.ratio=ratio	
    def recommend_wines(self,user_id, top_n=5,returnall=False):
        contentBasedWines=self.contentBasedModel.recommend_wines(user_id, top_n,returnall)
        collaborativeFilteringWines=self.collaborativeFilteringModel.recommend_wines(user_id, top_n,returnall)
        allWines=ratio*contentBasedWines+(1-ratio)*collaborativeFilteringWines
        return allWines
    
    

                                                                  